# Feature Engineering. Практическая работа

Сдаётся на проверку.

## Цели практической работы

Научиться: 

*   генерировать новые признаки; 
*   нормализовать и кодировать данные с помощью lambda-функций и библиотеки scikit-learn.


## Что входит в практическую работу

1. Построить признаки из строк.
2. Преобразовать категориальные переменные.
3. Стандартизировать данные.
4. Использовать MinMaxScaler для нормализации данных.
5. Создать признаки на основе дат.
6. Сохранить датафрейм для этапа моделирования.

## Что оценивается
- Программа выдаёт верный ответ на заданном наборе данных.
- Описаны причины выбранного решения, если требуется.
- Код читабелен: переменным даны осмысленные названия, соблюдены отступы и правила расстановки пробелов.
- При построении графиков название и оси подписаны, выведена легенда, если требуется. 
- Репозиторий проекта содержит осмысленные коммиты, содержащие конкретные реализованные фичи, ветки названы согласно назначению, лишние файлы не хранятся в репозитории.
- В репозитории проекта соблюдена иерархия директорий согласно списку модулей и содержащихся в них тем.
- Репозиторий проекта содержит файлы с данными, полученными в результате выполнения практической работы.


## Как отправить работу на проверку
Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему проверяющему куратору.


## Задача

Мы продолжим исследовать и готовить данные для этапа моделирования.

Вам предстоит работать с небольшой выборкой из коллекции подержанных автомобилей, выставленных на продажу в Соединённых Штатах, представленной в файле `data/vehicles_dataset_upd.csv`. На этих данных вы построите первую модель классификации, определяющую категорию цены подержанного автомобиля в зависимости от характеристик транспортного средства.

В этой практической работе вы продолжите генерировать дополнительные признаки и завершите подготовку датасета для этапа моделирования.

## Описание датасета:
- `id`— идентификатор записи;
- `url`— URL-записи о продаже;
- `region`— регион;
- `region_url`— URL региона;
- `price`— стоимость;
- `year`— год выпуска;
- `manufacturer`— производитель;
- `model`— модель;
- `condition`— состояние;
- `cylinders`— количество цилиндров;
- `fuel`— тип топлива;
- `odometer`— количество пройденных миль;
- `title_status`— статус;
- `transmission`— коробка передач;
- `VIN`— идентификационный номер;
- `drive`— тип привода;
- `size`— размер;
- `type`— кузов;
- `paint_color`— цвет;
- `image_url`— URL изображения;
- `description`— указанное описание;
- `county`— страна;
- `state`— штат;
- `lat`— широта;
- `long`— долгота;
- `posting_date`— дата размещения объявления о продаже;
- `price_category`— категория цены.


In [2]:
# Импортируйте необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

### Задача 1. Построение признаков из строк

1. Загрузите датасет из `data/vehicles_dataset_upd6.csv`, выведите первые пять записей и список колонок.

In [3]:
# Ваш код здесь
df=pd.read_csv("E:/Sunkar/interesting/Python/data/vehicles_dataset_upd6.csv",sep=',')
print(df.head())
df.columns

           id                                                url  \
0  7308295377  https://chattanooga.craigslist.org/ctd/d/chatt...   
1  7316380095  https://newjersey.craigslist.org/ctd/d/carlsta...   
2  7313733749  https://reno.craigslist.org/ctd/d/atlanta-2017...   
3  7308210929  https://fayetteville.craigslist.org/ctd/d/rale...   
4  7303797340  https://knoxville.craigslist.org/ctd/d/knoxvil...   

         region                           region_url  price  year  \
0   chattanooga   https://chattanooga.craigslist.org  54990  2020   
1  north jersey     https://newjersey.craigslist.org  16942  2016   
2  reno / tahoe          https://reno.craigslist.org  35590  2017   
3  fayetteville  https://fayetteville.craigslist.org  14500  2013   
4     knoxville     https://knoxville.craigslist.org  14590  2012   

  manufacturer                   model    fuel  odometer  ...  \
0          ram  2500 crew cab big horn  diesel     27442  ...   
1         ford    explorer 4wd 4dr xlt   other

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer', 'title_status', 'transmission',
       'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
       'price_category', 'date', 'odometer_km', 'odometer/price', 'region_new',
       'region_corrected', 'manufacturer_model', 'desc_len', 'model_in_desc',
       'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid', 'x0_other',
       'std_scaled_odometer', 'std_scaled_price'],
      dtype='object')

2. Выведите список значений и количество уникальных значений для модели автомобиля.

In [4]:
# Ваш код здесь
df['model'].value_counts()

model
f-150                        145
other                        107
silverado 1500               105
1500                          99
camry                         76
                            ... 
sierra1500 crewcab 4x4         1
corolla se                     1
xc60 t6 inscription sport      1
silverado 1500 ls              1
a3 2.0t premium plus pzev      1
Name: count, Length: 3341, dtype: int64

3. Количество уникальных значений велико. Попробуем их сократить, оставив только первое слово в наименовании модели. С помощью lambda-функции и преобразования над строками создайте новый признак `short_model`, содержащий только первое слово из наименования модели, выведите его и количество получившихся уникальных значений.

In [5]:
# Ваш код здесь
df['short_model']=df['model'].apply(lambda x: x.split(' ')[0])
df['short_model'].value_counts()

short_model
silverado    392
1500         205
wrangler     204
grand        201
f-150        197
            ... 
sle            1
classuc        1
s3             1
cts-v          1
gs350          1
Name: count, Length: 943, dtype: int64

### Задача 2. Преобразование категориальных переменных

1. Для категориальных переменных `short_model`, `transmission`, `region`, `manufacturer`, `state`, `title_status`, `age_category` распечатайте наименование категории и количество уникальных значений в ней.


In [6]:
# Ваш код здесь
df[['short_model','transmission','region','manufacturer','state','title_status']].nunique()

short_model     943
transmission      3
region          393
manufacturer     40
state            51
title_status      6
dtype: int64

2. Создайте датафрейм `data`, содержащий только выбранные категориальные переменные.

In [7]:
# Ваш код здесь
data=df[['short_model','transmission','region','manufacturer','state','title_status']]
data.describe()

,short_model,transmission,region,manufacturer,state,title_status
count,9619,9619,9619,9619,9619,9619
unique,943,3,393,40,51,6
top,silverado,automatic,"kansas city, MO",ford,ca,clean
freq,392,7363,86,1561,1073,9314


3. Создайте объект класса OneHotEncoder, который будет отвечать за преобразование. Уточните, что на выходе должна быть неразреженная матрица.

In [8]:
# Ваш код здесь
ohe=OneHotEncoder(sparse_output=False)
ohe

OneHotEncoder(sparse_output=False)

4. Идентифицируйте параметры преобразования на датафрейме с категориальными переменными.

In [9]:
# Ваш код здесь
ohe.fit(data)

OneHotEncoder(sparse_output=False)

5. Примените преобразование для датафрейма с категориальными переменными и сохраните в новую переменную, распечатайте получившуюся матрицу и её размерность.

In [10]:
# Ваш код здесь
ohe_data=ohe.transform(data)
ohe_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

6. Выведите новые наименования признаков с помощью метода `get_feature_names_out`.

In [11]:
# Ваш код здесь
ohe.get_feature_names_out()

array(['short_model_-benz', 'short_model_1', 'short_model_124', ...,
       'title_status_parts only', 'title_status_rebuilt',
       'title_status_salvage'], dtype=object)

7. Добавьте в исходный датафрейм получившиеся новые признаки, выведите первые пять записей датафрейма и список колонок.

In [12]:
# Ваш код здесь
df[ohe.get_feature_names_out()]=ohe_data
df.columns

C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\2863843115.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_data
C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\2863843115.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()]=ohe_data
C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\2863843115.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'state_wa', 'state_wi', 'state_wv', 'state_wy', 'title_status_clean',
       'title_status_lien', 'title_status_missing', 'title_status_parts only',
       'title_status_rebuilt', 'title_status_salvage'],
      dtype='object', length=1471)

8. Посмотрите на результат обратного преобразования получившейся матрицы в исходные признаки, используйте `inverse_transform`.

In [13]:
# Ваш код здесь
ohe.inverse_transform(ohe_data)

array([['2500', 'other', 'chattanooga', 'ram', 'tn', 'clean'],
       ['explorer', 'automatic', 'north jersey', 'ford', 'nj', 'clean'],
       ['golf', 'other', 'reno / tahoe', 'volkswagen', 'ca', 'clean'],
       ...,
       ['a3', 'automatic', 'salem', 'audi', 'or', 'clean'],
       ['cayenne', 'automatic', 'madison', 'porsche', 'wi', 'clean'],
       ['1500', 'other', 'norfolk / hampton roads', 'ram', 'va', 'clean']],
      dtype=object)

### Задача 3. Стандартизация данных

1. Создайте переменную под количественные переменные `lat`, `long`, `year`, `odometer/price`, `desc_len`, `model_in_desc`, `model_len`, `model_word_count` и создайте датафрейм `data`, содержащий только выбранные количественные переменные.


In [14]:
# Ваш код здесь
data=df[['lat','long','year','odometer/price','desc_len','model_in_desc']]
data

,lat,long,year,odometer/price,desc_len,model_in_desc
0,35.060000,-85.250000,2020,0.499036,4482,0
1,40.821805,-74.061962,2016,3.542852,968,0
2,33.779214,-84.411811,2017,0.394718,4286,0
3,35.715954,-78.655304,2013,8.089034,3241,0
4,35.970000,-83.940000,2012,5.515079,4851,0
...,...,...,...,...,...,...
9614,42.123900,-79.189500,2002,33.370412,1710,0
9615,43.216990,-77.755610,2008,7.835323,948,0
9616,44.925908,-122.982753,2011,16.696387,909,0
9617,43.029559,-89.397796,2015,1.941787,3644,0


2. Создайте объект `std_scaler` класса StandardScaler с параметрами по умолчанию, который будет отвечать за стандартизацию.

In [15]:
# Ваш код здесь
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
std_scaler

StandardScaler()

3. Идентифицируйте параметры преобразования на датафрейме с количественными переменными.

In [16]:
# Ваш код здесь
std_scaler.fit(data)
std_scaler

StandardScaler()

4. Примените преобразование для датафрейма с количественными переменными и сохраните в новую переменную, распечатайте получившуюся матрицу и её размерность.

In [17]:
# Ваш код здесь
std_scaled=std_scaler.transform(data)
print(std_scaled.shape)
std_scaled

(9619, 6)


array([[-0.61917248,  0.48424452,  1.32239376, -0.51078432,  0.63207479,
        -0.15578836],
       [ 0.38801429,  1.11079989,  0.69597272, -0.40294692, -0.64678136,
        -0.15578836],
       [-0.84305905,  0.531185  ,  0.85257798, -0.51448016,  0.56074417,
        -0.15578836],
       ...,
       [ 1.10542805, -1.62887477, -0.08705357,  0.06306122, -0.66825333,
        -0.15578836],
       [ 0.7739386 ,  0.25195859,  0.53936746, -0.45967004,  0.32709999,
        -0.15578836],
       [-0.33773799,  0.98546471,  0.85257798, -0.48868248,  0.63243873,
        -0.15578836]])

5. Создайте наименования новых признаков по шаблону: `lat_std` для `lat` и так далее.

In [18]:
# Ваш код здесь
for i in range(len(std_scaler.get_feature_names_out())):
    data[f'{std_scaler.get_feature_names_out()[i]}_std']=std_scaled[:,i:i+1]
data.head()

C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\4285504448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{std_scaler.get_feature_names_out()[i]}_std']=std_scaled[:,i:i+1]
C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\4285504448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{std_scaler.get_feature_names_out()[i]}_std']=std_scaled[:,i:i+1]
C:\Users\KCN_DASUTP_2025\AppData\Local\Temp\ipykernel_11720\4285504448.py:3: SettingWithCopyWarning: 
A value is trying 

,lat,long,year,odometer/price,desc_len,model_in_desc,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std
0,35.060000,-85.250000,2020,0.499036,4482,0,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788
1,40.821805,-74.061962,2016,3.542852,968,0,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788
2,33.779214,-84.411811,2017,0.394718,4286,0,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788
3,35.715954,-78.655304,2013,8.089034,3241,0,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788
4,35.970000,-83.940000,2012,5.515079,4851,0,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788


6. Добавьте в исходный датафрейм получившиеся новые признаки, выведите первые пять записей датафрейма и список колонок.

In [38]:
# Ваш код здесь
for i in range(len(std_scaler.get_feature_names_out())):
    df[f'{std_scaler.get_feature_names_out()[i]}_std']=std_scaled[:,i:i+1]
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,0.0,0.0,0.0,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,0.0,0.0,0.0,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,0.0,0.0,0.0,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,0.0,0.0,0.0,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,0.0,0.0,0.0,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788


### Задача 4. MinMax-нормализация данных

1. Импортируйте MinMaxScaler из `sklearn.preprocessing`.

In [20]:
# Ваш код здесь
from sklearn.preprocessing import MinMaxScaler

2. Создайте датафрейм, содержащий переменные `lat` и `long`.

In [40]:
# Ваш код здесь
df_lat_long=df[['lat','long']]
df_lat_long

,lat,long
0,35.060000,-85.250000
1,40.821805,-74.061962
2,33.779214,-84.411811
3,35.715954,-78.655304
4,35.970000,-83.940000
...,...,...
9614,42.123900,-79.189500
9615,43.216990,-77.755610
9616,44.925908,-122.982753
9617,43.029559,-89.397796


3. Повторите действия пунктов 2–6 из задачи 3 для добавления в исходный датафрейм признаков `lat_mm` и `long_mm`. Выведите получившийся датафрейм.

In [47]:
# Ваш код здесь
std_scaler=MinMaxScaler()
std_scaler.fit(df_lat_long)
std_scaled=std_scaler.transform(df_lat_long)
for i in range(len(std_scaler.get_feature_names_out())):
    df[f'{std_scaler.get_feature_names_out()[i]}_mm']=std_scaled[:,i:i+1]

df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'title_status_rebuilt', 'title_status_salvage', 'lat_std', 'long_std',
       'year_std', 'odometer/price_std', 'desc_len_std', 'model_in_desc_std',
       'lat_mm', 'long_mm'],
      dtype='object', length=1479)

4. Создайте переменную под географические признаки `lat`, `lat_std`, `lat_mm`, `long`, `long_std`, `long_mm` и выведите их. Посмотрите на различия в значениях для исходных признаков и нормализованных с помощью стандартизации и MinMax-преобразования. 

In [48]:
# Ваш код здесь
std_mm_sign=df[['lat','lat_std','lat_mm','long','long_std','long_mm']]
std_mm_sign

,lat,lat_std,lat_mm,long,long_std,long_mm
0,35.060000,-0.619172,0.341769,-85.250000,0.484245,0.288699
1,40.821805,0.388014,0.468665,-74.061962,1.110800,0.333055
2,33.779214,-0.843059,0.313561,-84.411811,0.531185,0.292022
3,35.715954,-0.504509,0.356215,-78.655304,0.853562,0.314844
4,35.970000,-0.460101,0.361810,-83.940000,0.557607,0.293893
...,...,...,...,...,...,...
9614,42.123900,0.615626,0.497341,-79.189500,0.823646,0.312727
9615,43.216990,0.806702,0.521415,-77.755610,0.903947,0.318411
9616,44.925908,1.105428,0.559052,-122.982753,-1.628875,0.139104
9617,43.029559,0.773939,0.517287,-89.397796,0.251959,0.272255


5. Выведите описательные статистики для всех географических признаков. Посмотрите на различия в статистиках для исходных признаков и нормализованных с помощью стандартизации и MinMax-преобразования.

In [49]:
# Ваш код здесь
std_mm_sign.describe()

,lat,lat_std,lat_mm,long,long_std,long_mm
count,9619.000000,9.619000e+03,9619.000000,9619.000000,9.619000e+03,9619.000000
mean,38.602095,2.105257e-16,0.419779,-93.896875,-4.498602e-16,0.254418
std,5.720989,1.000052e+00,0.125997,17.857352,1.000052e+00,0.070797
min,19.541726,-3.331829e+00,0.000000,-158.069300,-3.593801e+00,0.000000
25%,34.920000,-6.436450e-01,0.338685,-106.944340,-7.306875e-01,0.202690
50%,39.300000,1.219966e-01,0.435149,-87.845800,3.388739e-01,0.278408
75%,42.382100,6.607601e-01,0.503028,-80.678711,7.402470e-01,0.306822
max,64.947500,4.605283e+00,1.000000,94.163200,1.053179e+01,1.000000


6. Выведите матрицу корреляций для всех географических признаков, чтобы убедиться, что нормализация признаков сохранила свойства исходных признаков.

In [51]:
# Ваш код здесь
df['lat'].corr(df['lat_std'])

0.9999999999999993

### Задача 5. Признаки на основе дат

1. Выведите по отдельности колонки `posting_date` и `date`. 

In [54]:
# Ваш код здесь
print(df['posting_date'])
print(df['date'])

0       2021-04-17T12:30:50-0400
1       2021-05-03T15:40:21-0400
2       2021-04-28T03:52:20-0700
3       2021-04-17T10:08:57-0400
4       2021-04-08T15:10:56-0400
                  ...           
9614    2021-04-10T16:33:57-0400
9615    2021-05-03T09:36:30-0400
9616    2021-04-22T12:14:01-0700
9617    2021-04-14T09:14:42-0500
9618    2021-04-24T13:50:49-0400
Name: posting_date, Length: 9619, dtype: object
0       2021-04-17 16:30:50+00:00
1       2021-05-03 19:40:21+00:00
2       2021-04-28 10:52:20+00:00
3       2021-04-17 14:08:57+00:00
4       2021-04-08 19:10:56+00:00
                  ...            
9614    2021-04-10 20:33:57+00:00
9615    2021-05-03 13:36:30+00:00
9616    2021-04-22 19:14:01+00:00
9617    2021-04-14 14:14:42+00:00
9618    2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: object


2. Ранее преобразованная колонка `date` после сохранения в файл и загрузки из него содержит данные в строковом формате, поэтому измените её тип c помощью `pd.to_datetime`.

In [59]:
# Ваш код здесь
df['date']=pd.to_datetime(df['date'])
df['date']

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-04-08 19:10:56+00:00
                  ...           
9614   2021-04-10 20:33:57+00:00
9615   2021-05-03 13:36:30+00:00
9616   2021-04-22 19:14:01+00:00
9617   2021-04-14 14:14:42+00:00
9618   2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: datetime64[ns, UTC]

3. Несколькими способами извлеките номер месяца продажи автомобиля.

3.1. Из переменной `posting_date` с помощью lambda-функции и операций над строками.

In [57]:
# Ваш код здесь
df['posting_date'].apply(lambda x: x.split('-')[1])

0       04
1       05
2       04
3       04
4       04
        ..
9614    04
9615    05
9616    04
9617    04
9618    04
Name: posting_date, Length: 9619, dtype: object

3.2. Из переменной `date` с помощью lambda-функции и метода `month`.

In [64]:
# Ваш код здесь
df['date'].apply(lambda x: x.month)

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: date, Length: 9619, dtype: int64

3.3. Из переменной `date` с помощью `dt` и метода `month`.

In [68]:
# Ваш код здесь
df['date'].dt.month

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: date, Length: 9619, dtype: int32

4. Создайте любым из способов (3.2 или 3.3) признак `month`, обозначающий номер месяца размещения объявления о продаже автомобиля, и распечатайте его.

In [73]:
# Ваш код здесь
df['month']=df['date'].dt.month
df['month']

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: month, Length: 9619, dtype: int32

5. Создайте признак `dayofweek`, обозначающий день недели размещения объявления о продаже автомобиля, и распечатайте его.

In [74]:
# Ваш код здесь
df['dayofweek']=df['date'].dt.weekday
df['dayofweek']

0       5
1       0
2       2
3       5
4       3
       ..
9614    5
9615    0
9616    3
9617    2
9618    5
Name: dayofweek, Length: 9619, dtype: int32

6. Создайте признак `diff_years`, обозначающий количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля, и распечатайте его. Дополнительные корректировки не применяйте.

In [76]:
# Ваш код здесь
df['diff_year']=df['date'].dt.year-df['year']
df['diff_year']

0        1
1        5
2        4
3        8
4        9
        ..
9614    19
9615    13
9616    10
9617     6
9618     4
Name: diff_year, Length: 9619, dtype: int64

7. Примените стандартизацию к новым признаками и сохраните результат в новые признаки вида `month_std`.

In [86]:
# Ваш код здесь
data=df[['month','dayofweek','diff_year']]
std_scaler=StandardScaler()
std_scaler.fit(data)
std_scaled=std_scaler.transform(data)
for i in range(len(std_scaler.get_feature_names_out())):
    df[f'{std_scaler.get_feature_names_out()[i]}_std']=std_scaled[:,i:i+1]
df.head()
print(df.columns.tolist())

['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model', 'fuel', 'odometer', 'title_status', 'transmission', 'image_url', 'description', 'state', 'lat', 'long', 'posting_date', 'price_category', 'date', 'odometer_km', 'odometer/price', 'region_new', 'region_corrected', 'manufacturer_model', 'desc_len', 'model_in_desc', 'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid', 'x0_other', 'std_scaled_odometer', 'std_scaled_price', 'short_model', 'short_model_-benz', 'short_model_1', 'short_model_124', 'short_model_128i', 'short_model_135i', 'short_model_1500', 'short_model_163869', 'short_model_1928', 'short_model_1951', 'short_model_1954', 'short_model_1960s', 'short_model_1966', 'short_model_1968', 'short_model_1972', 'short_model_1977', 'short_model_1978', 'short_model_198.5', 'short_model_1988', 'short_model_1994', 'short_model_1999', 'short_model_2', 'short_model_200', 'short_model_2000', 'short_model_2002', 'short_model_2006', 'short_model_2007', 'short_model_2008'

### Задача 6. Сохранение датафрейма для этапа моделирования

1. Удалите колонки, которые были исходными для формирования признаков, а также новые признаки, которые были добавлены с целью обучения.  Сохраните новый датафрейм в переменную `df_prepared`.

In [88]:
# Допишите код
column_for_del=['url','region','region_url','id','desc_len','lat_mm', 'long_mm','month', 'dayofweek', 
                'diff_year', 'lat', 'long', 'short_model','model_in_desc','date', 'std_scaled_price','odometer/price_std']
df.drop(columns=column_for_del,axis=1,inplace=True)
df.shape

(9619, 1468)

2. Распечатайте оставшийся список колонок и размерность финального датафрейма.

In [89]:
# Ваш код здесь
print(df.columns.tolist())

['price', 'year', 'manufacturer', 'model', 'fuel', 'odometer', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date', 'price_category', 'odometer_km', 'odometer/price', 'region_new', 'region_corrected', 'manufacturer_model', 'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid', 'x0_other', 'std_scaled_odometer', 'short_model_-benz', 'short_model_1', 'short_model_124', 'short_model_128i', 'short_model_135i', 'short_model_1500', 'short_model_163869', 'short_model_1928', 'short_model_1951', 'short_model_1954', 'short_model_1960s', 'short_model_1966', 'short_model_1968', 'short_model_1972', 'short_model_1977', 'short_model_1978', 'short_model_198.5', 'short_model_1988', 'short_model_1994', 'short_model_1999', 'short_model_2', 'short_model_200', 'short_model_2000', 'short_model_2002', 'short_model_2006', 'short_model_2007', 'short_model_2008', 'short_model_2009', 'short_model_2012', 'short_model_2015', 'short_model_2016', 'short_model_2017', 'short_model_2018', 's

3. Запишите преобразованный датафрейм в файл `data/vehicles_dataset_prepared.csv`, индекс не сохраняйте.

In [91]:
# Ваш код здесь
df.to_csv('E:/Sunkar/interesting/Python/data/vehicles_dataset_prepared.csv',index=False)

## Описание преобразованного датасета:
- `id`— идентификатор записи;
- `is_manufacturer_name`— признак производителя автомобиля;

- `region_*`— регион;
- `x0_*`— тип топлива;
- `manufacturer_*`— производитель;
- `short_model_*`— сокращённая модель автомобиля;
- `title_status_*`— статус;
- `transmission_*`— коробка передач;
- `state_*`— штат;
- `age_category_*`— возрастная категория автомобиля;

- `std_scaled_odometer`— количество пройденных миль (после стандартизации);
- `year_std`— год выпуска (после стандартизации);
- `lat_std`— широта (после стандартизации);
- `long_std`— долгота (после стандартизации);
- `odometer/price_std`— отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`— количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`— количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`— длина наименования автомобиля (после стандартизации);
- `model_word_count_std`— количество слов в наименовании автомобиля (после стандартизации);
- `month_std`— номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`— день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`— количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`— стоимость;
- `price_category`– категория цены.